In [6]:
import numpy as np
import pandas as pd
import itertools
import os
import matplotlib.pyplot as plt

localpath_prefix='/Users/haizhao/OneDrive - Columbia University Irving Medical Center' 


In [8]:
#@title Load Ecoli_ZEPPI data files; takes 3s~ after files are ready.

filetoread1=localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/ecoli_af_lrg1_ZIPPI_05222023.csv'
filetoread2=localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/ecoli_af_lrl1_ZIPPI_05222023_nRd.csv'
df_g1 = pd.read_csv(filetoread1,sep=',',usecols=['Uni_pair','LR','Zmaxmt'])
df_l1 = pd.read_csv(filetoread2,sep=',',usecols=['Uni_pair','LR','Zmaxmt'])

df_zp_ecl = pd.concat([df_g1, df_l1], ignore_index=True)
df_zp_ecl = df_zp_ecl.reset_index(drop=True)

print("Number of total_PPIs from ZEPPI:",len(df_zp_ecl))


Number of total_PPIs from ZEPPI: 5628400


In [9]:
# Read DS predictions; ~4s

filetoread=localpath_prefix+'/WORK/PrePPI_MI/DS/K12/K12_full.tsv'
#filetoread=localpath_prefix+'/WORK/PrePPI_MI/DS/K12_v1/honig_ecoli_predictions.tsv'
df_DS=pd.read_csv(filetoread,sep='\t',names=["Uniprot1", "Uniprot2",'DS'])
df_DS = df_DS.dropna()
df_DS['Uni_pair']=df_DS['Uniprot1']+"_"+ df_DS['Uniprot2']
df_DS_ecl=df_DS[['Uni_pair','DS']]

print("Number of total_PPIs from DS:",len(df_DS_ecl))


Number of total_PPIs from DS: 9638235


In [10]:
#--- Read PrePPI predictions (PrePPI_af), the non-structure ones; ~4s

def readPrePPIcsv_returncsv_sortedpair_score(filename,sep_symbol='\t',skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN, usecols=[0,1,2])
    df0.columns=["UniprotID_A", "UniprotID_B","SMscore"]
    df0['Uni_pair']= df0['UniprotID_A'].astype(str)+'_' +df0['UniprotID_B'].astype(str)
    return df0

def readdf_returndict_score(df0,lr_cut=0):
    if lr_cut==0:
        return  dict(zip(df0['Uni_pair'],df0['SMscore']))
    else:
        df1 = df0[(df0['SMscore'].astype(float) > int(lr_cut))]
        return dict(zip(df1['Uni_pair'],df1['SMscore']))

pred_af_lr0_nonstr_df = readPrePPIcsv_returncsv_sortedpair_score(localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/ecoli_af.lr0.4col.nonstructure', sep_symbol=',')
pred_af_lr0_nonstr_dict = readdf_returndict_score(pred_af_lr0_nonstr_df,lr_cut=0);

print(len(pred_af_lr0_nonstr_df))

def readcsv6_returncsv_sortedpair_score(filename,sep_symbol='\t',skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN, usecols=[0,1,4])
    df0.columns=["UniprotID_A", "UniprotID_B","SMscore"]
    df0['UniA_UniB']= df0['UniprotID_A'].astype(str)+'_' +df0['UniprotID_B'].astype(str)
    return df0

#pred_af_lr0_df =readcsv6_returncsv_sortedpair_score(localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/ecoli_af.lr0.6col.csv',sep_symbol=' ')
#pred_af_lr0_df = pred_af_lr0_df.drop_duplicates()
#pred_af_lr0_df_SM = pred_af_lr0_df['SMscore'].tolist()
#pred_af_lr0_df_dict = readdf_returndict_score(pred_af_lr0_df,lr_cut=0);


5816242


In [12]:
def readcsv6_returncsv_sortedpair_score(filename,sep_symbol='\t',skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN, usecols=[0,1,4])
    df0.columns=["UniprotID_A", "UniprotID_B","SMscore"]
    df0['Uni_pair']= df0['UniprotID_A'].astype(str)+'_' +df0['UniprotID_B'].astype(str)
    return df0

pred_af_lr0_df =readcsv6_returncsv_sortedpair_score(localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/ecoli_af.lr0.6col.csv',sep_symbol=' ')
pred_af_lr0_df = pred_af_lr0_df.drop_duplicates()
pred_af_lr0_df_SM = pred_af_lr0_df['SMscore'].tolist()
pred_af_lr0_df_dict = readdf_returndict_score(pred_af_lr0_df,lr_cut=0);

In [14]:
#--- Read E.coli HINT HQ-LC and allDBs; 

def readcsv2_return_sortedpair(filename,sep_symbol,skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN) #names=["hfpd_ID", "Uni"]
    ref_pairs=tuple(zip(df0['Uniprot_A'], df0['Uniprot_B']))
    ref_pairs_sorted=[]
    for i in ref_pairs:
        i = '_'.join(sorted(i))
        if i not in ref_pairs_sorted:
            ref_pairs_sorted.append(i)
    return ref_pairs_sorted

def readcsv_Int3D_return_sortedpair(filename,sep_symbol,skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN) #names=["hfpd_ID", "Uni"]
    ref_pairs=tuple(zip(df0['PROT1'], df0['PROT2'])) #PROT1,PROT2
    ref_pairs_sorted=[]
    for i in ref_pairs:
        i = '_'.join(sorted(i))
        if i not in ref_pairs_sorted:
            ref_pairs_sorted.append(i)
    return ref_pairs_sorted

HINT_binary_hq_LC_pairs = readcsv2_return_sortedpair(localpath_prefix+'/WORK/Ecoli_MI/Expts/2021HINT_EColiK12_binary_hq_LC.txt','\t')
df_hqLC = pd.DataFrame(HINT_binary_hq_LC_pairs, columns=['Uni_pair'])

Int3D_all_pairs = readcsv_Int3D_return_sortedpair(localpath_prefix+'/WORK/Ecoli_MI/Expts/2021Interactome3d_complete.csv',',')
df_Int3Dall = pd.DataFrame(Int3D_all_pairs, columns=['Uni_pair'])

#--- Read allExpDBs database

allExpDB_file = localpath_prefix+'/WORK/Ecoli_MI/Expts/EcoliK12_ExpDBs_all.csv'
allExpDB_df = pd.read_csv(allExpDB_file,skiprows=2,usecols=[0],names=["Uni_pair"])
allExpDB_list = list(set(allExpDB_df['Uni_pair']))

print("Number of Pos_set PPIs:",len(df_hqLC))
print("Number of Int3Dall PPIs:",len(df_Int3Dall))

print("Number of allExpDB_PPIs:",len(allExpDB_df))

#--- Read STRING database
def readcsv_return_sortedpair(filename,sep_symbol=',',skipN=0):
    df0 = pd.read_csv(filename,sep=sep_symbol,skiprows=skipN,usecols=['protein1','protein2','experimental','combined_score']) 
    ref_pairs=tuple(zip(df0['protein1'], df0['protein2']))
    ref_pairs_sorted=[]
    for i in ref_pairs:
        i = '_'.join(sorted(i))
        ref_pairs_sorted.append(i)
    return ref_pairs_sorted

STRING_K12_pairs_physical_exp = readcsv_return_sortedpair(localpath_prefix+'/WORK/Ecoli_MI/Expts/Ecoli/STRING_ecoli.K12MG1655/511145_83333.physical.links.experimental.nonred.txt')
print("The # of STRING K12 (physical; exp >0)",len(STRING_K12_pairs_physical_exp))
df_STRING = pd.DataFrame(STRING_K12_pairs_physical_exp, columns=['Uni_pair'])


Number of Pos_set PPIs: 1957
Number of Int3Dall PPIs: 1657
Number of allExpDB_PPIs: 565007
The # of STRING K12 (physical; exp >0) 10577


In [15]:
# Get E.coli  all-to-all PPIs; ~10s
from itertools import combinations

df0=pd.read_csv(localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/map_list',sep='\t>',names=["hfpd_ID", "Uni"],skiprows=0,dtype={'hfpd_ID': object})
ecoliID_dict = dict(zip(df0['hfpd_ID'],df0['Uni']))

def pairwiselists(arr, r):
    return [x[0]+'_'+x[1] for x in combinations(arr, r)]

K12_uni_all = list(df0['Uni'])
K12_pairs_all = pairwiselists(K12_uni_all,2)

def flatten(t):
    return [item for sublist in t for item in sublist.split('_')]
def intersection(lst1, lst2):
    return set(lst1).intersection(set(lst2))

allunknown_ref_list = set(K12_pairs_all) - set(allExpDB_df['Uni_pair'])
print(len(allunknown_ref_list),len(K12_pairs_all))

/var/folders/g9/763yp4f55l12h76s_v2530480000gq/T/ipykernel_81067/3434087374.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df0=pd.read_csv(localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/map_list',sep='\t>',names=["hfpd_ID", "Uni"],skiprows=0,dtype={'hfpd_ID': object})


9075032 9638245


In [ ]:
#@title Import Bayes models that were trained on Human

import pickle
with open('../data/human_ZP_Bayes_ratio1000.pkl', 'rb') as f:
    nbn_ZP = pickle.load(f)

with open('../data/human_DS_Bayes_STRING.pkl', 'rb') as f:
    nbn_DS = pickle.load(f)


In [18]:
#@title Apply the trained Bayes model to E. coli; takes ~1min

df_zp_ecl['LR_ZP'] = nbn_ZP.predict(df_zp_ecl['Zmaxmt'].to_numpy().reshape(len(df_zp_ecl),1))
df_DS_ecl['LR_DS'] = nbn_DS.predict(df_DS_ecl['DS'].to_numpy().reshape(len(df_DS_ecl),1))


/var/folders/g9/763yp4f55l12h76s_v2530480000gq/T/ipykernel_81067/1376985900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_DS_ecl['LR_DS'] = nbn_DS.predict(df_DS_ecl['DS'].to_numpy().reshape(len(df_DS_ecl),1))


In [ ]:
#@title Combine three different clues; fill the missing LR as 1; ~40s

# Merge all three DataFrames
df_threeclues = pred_af_lr0_nonstr_df.merge(df_DS_ecl, on='Uni_pair', how='outer').merge(df_zp_ecl, on='Uni_pair', how='outer')

# Fill NaN values with 1
df_threeclues.fillna({'SMscore':1}, inplace=True)
df_threeclues.fillna({'LR_DS':1}, inplace=True)
df_threeclues.fillna({'LR_ZP':1}, inplace=True)

# Compute the total score
df_threeclues['LR_total'] = df_threeclues['SMscore'] * df_threeclues['LR_DS'] * df_threeclues['LR_ZP']
df_threeclues['LR_SMDS'] = df_threeclues['SMscore'] * df_threeclues['LR_DS']
df_threeclues['LR_ZPDS'] = df_threeclues['LR_ZP'] * df_threeclues['LR_DS']
df_threeclues['LR_SMZP'] = df_threeclues['SMscore'] * df_threeclues['LR_ZP']

print(len(df_threeclues))


In [12]:
df_threeclues_out=df_threeclues[['Uni_pair','SMscore','DS','LR','Zmaxmt','LR_ZP', 'LR_DS','LR_total', 'LR_SMDS', 'LR_ZPDS', 'LR_SMZP']]

In [ ]:
filewrite5=localpath_prefix+'/WORK/PrePPI_MI/Ecoli_af/threeclues_LRtotal.csv'

df_threeclues_out.to_csv(filewrite5,index=False)